In [ ]:
# !pip install LightGBM

In [1]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error 

In [ ]:
param = {'num_leaves': 31, 'objective': 'mean_squared_error', 'boosting':'gbdt', 'num_iterations': 100,'learning_rate': 0.1, 'max_depth': 3, 'min_data_in_leaf': 10, 'bagging_fraction' : 0.1,
'bagging_freq' : 10, 'early_stopping_round' : 50, 'lambda_l1': 0.06, 'lambda_l2' : 0.01 }
param['metric'] = 'rmse'

In [ ]:
X_train = np.load( 'data/X_train_1.npy',allow_pickle=True)
y_train = np.load( 'data/y_train_1.npy',allow_pickle=True)
X_val = np.load('data/X_val.npy',allow_pickle=True)
y_val = np.load('data/y_val.npy',allow_pickle=True)
X_val1 = np.load( 'data/X_val1_1.npy',allow_pickle=True)
y_val1 = np.load( 'data/y_val1_1.npy',allow_pickle=True)
X_val2 = np.load( 'data/X_val2_1.npy',allow_pickle=True)
y_val2 = np.load('data/y_val2_1.npy', allow_pickle=True)
X_train.shape, X_val.shape, y_train.shape, y_val.shape, X_val1.shape, X_val2.shape, y_val1.shape, y_val2.shape

In [2]:
X_train = np.load( 'data/X_train_2.npy',allow_pickle=True)
y_train = np.load( 'data/y_train_2.npy',allow_pickle=True)
X_val = np.load('data/X_val_2.npy',allow_pickle=True)
y_val = np.load('data/y_val_2.npy',allow_pickle=True)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((76500, 166), (19125, 166), (76500,), (19125,))

In [2]:
X_train = np.load( 'data/X_train_ungrouped.npy',allow_pickle=True)
y_train = np.load( 'data/y_train_ungrouped.npy',allow_pickle=True)
X_val = np.load('data/X_val_ungrouped.npy',allow_pickle=True)
y_val = np.load('data/y_val_ungrouped.npy',allow_pickle=True)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((382992, 214), (95749, 214), (382992,), (95749,))

In [3]:
d_train = lgb.Dataset(X_train, label=y_train, silent=True)
d_val = lgb.Dataset(X_val, label=y_val, silent=True)

In [5]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape,# X_val1.shape, X_val2.shape, y_val1.shape, y_val2.shape

((76500, 166), (19125, 166), (76500,), (19125,))

In [ ]:
# bst = lgb.train(param, d_train, valid_sets=[d_val])
# train_predict = bst.predict(X_val, num_iteration=bst.best_iteration)

In [4]:
import optuna
import logging
import sys
import os

optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))

study_name = "lgb-study_ver3"

storage_name = "sqlite:///{}/{}.db".format(os.getcwd(), study_name) 





In [ ]:
optuna.delete_study(study_name, storage_name)

In [5]:

study = optuna.create_study(
study_name=study_name,
storage=storage_name,
load_if_exists=True,
direction="minimize",
)

[I 2022-03-28 12:45:27,332] A new study created in RDB with name: lgb-study_ver3


A new study created in RDB with name: lgb-study_ver3


In [6]:

def objective(trial):

    param = {
        'verbosity':-1,
        # 'num_leaves': trial.suggest_int("num_leaves", 1, 20,),
        'objective': 'mean_squared_error', 
        'boosting':trial.suggest_categorical("boosting", ["gbdt"]),# "rf",]),#"dart"]), 
        'num_iterations': 1000,  
        'learning_rate': trial.suggest_float("learning_rate", 1e-5, 1),
        'max_depth':trial.suggest_int("max_depth", 3, 5,),
        # 'min_data_in_leaf': trial.suggest_int("min_data_in_leaf",1,10,),
        'bagging_fraction' : trial.suggest_float("bagging_fraction", 0.1, 1.0),
        'bagging_freq' : trial.suggest_int("bagging_freq",20, 40),
        'early_stopping_round' : 50, 
        'lambda_l1': trial.suggest_float("lambda_l1", 1e-5, 10, log=True), 
        'lambda_l2' : trial.suggest_float("lambda_l2", 1e-5, 20 , log=True), 
        'metric': 'rmse',
    #     'feature_pre_filter' : False
    }

    lgb_model = lgb.train(
        params = param,
        train_set = d_train,
        valid_sets=[d_val],
        )
    
    if param["boosting"] in  ["gbdt", "dart"]:
        trial.set_user_attr('best_iteration', lgb_model.best_iteration)
        train_predict = lgb_model.predict(X_train, num_iteration=lgb_model.best_iteration)
        val_predict = lgb_model.predict(X_val, num_iteration=lgb_model.best_iteration)
        # val2_predict = xgb_model.predict(d_val2, ntree_limit=xgb_model.best_ntree_limit)
    else:
        train_predict = lgb_model.predict(X_train)
        val_predict = lgb_model.predict(X_val)

    train_rmse = np.sqrt(mean_squared_error(y_train, train_predict))
    val_rmse = np.sqrt(mean_squared_error(y_val, val_predict))
    # val2_rmse = np.sqrt(mean_squared_error(y_val2, val2_predict)))

    trial.set_user_attr('train_rmse',train_rmse)
    trial.set_user_attr('val_rmse',val_rmse)

    return val_rmse


In [9]:
study.optimize(objective, n_trials = 100, show_progress_bar=True)

trials_df = study.trials_dataframe()

trials_df.to_excel(study_name + '.xlsx') 

C:\Users\Haripriya\anaconda3\lib\site-packages\optuna\progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()
  0%|          | 0/100 [00:00<?, ?it/s]C:\Users\Haripriya\anaconda3\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[1]	valid_0's rmse: 0.816397
Training until validation scores don't improve for 50 rounds
[2]	valid_0's rmse: 0.815435
[3]	valid_0's rmse: 0.814685
[4]	valid_0's rmse: 0.814174
[5]	valid_0's rmse: 0.813723
[6]	valid_0's rmse: 0.813388
[7]	valid_0's rmse: 0.813103
[8]	valid_0's rmse: 0.812865
[9]	valid_0's rmse: 0.812714
[10]	valid_0's rmse: 0.81252
[11]	valid_0's rmse: 0.812412
[12]	valid_0's rmse: 0.81231
[13]	valid_0's rmse: 0.812249
[14]	valid_0's rmse: 0.812174
[15]	valid_0's rmse: 0.81213
[16]	valid_0's rmse: 0.812058
[17]	valid_0's rmse: 0.812019
[18]	valid_0's rmse: 0.811995
[19]	valid_0's rmse: 0.811982
[20]	valid_0's rmse: 0.811968
[21]	valid_0's rmse: 0.811945
[22]	valid_0's rmse: 0.811938
[23]	valid_0's rmse: 0.811946
[24]	valid_0's rmse: 0.811929
[25]	valid_0's rmse: 0.811931
[26]	valid_0's rmse: 0.811929
[27]	valid_0's rmse: 0.811942
[28]	valid_0's rmse: 0.811953
[29]	valid_0's rmse: 0.811949
[30]	valid_0's rmse: 0.811945
[31]	valid_0's rmse: 0.811952
[32]	valid_0's rmse: 

  1%|          | 1/100 [00:01<02:09,  1.31s/it]

[I 2022-03-28 12:46:31,941] Trial 20 finished with value: 0.811891222560448 and parameters: {'boosting': 'gbdt', 'learning_rate': 0.14130769914036728, 'max_depth': 5, 'bagging_fraction': 0.5846321730187931, 'bagging_freq': 32, 'lambda_l1': 0.0007569623514577861, 'lambda_l2': 5.557155533409732}. Best is trial 6 with value: 0.8115680572614749.


C:\Users\Haripriya\anaconda3\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


...

[I 2022-03-28 12:50:47,707] Trial 118 finished with value: 0.8115401720091595 and parameters: {'boosting': 'gbdt', 'learning_rate': 0.1017263188808782, 'max_depth': 3, 'bagging_fraction': 0.7646702807348853, 'bagging_freq': 38, 'lambda_l1': 9.352315858569801, 'lambda_l2': 19.699695387117924}. Best is trial 30 with value: 0.8115003026285839.


C:\Users\Haripriya\anaconda3\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[1]	valid_0's rmse: 0.816638
Training until validation scores don't improve for 50 rounds
[2]	valid_0's rmse: 0.815792
[3]	valid_0's rmse: 0.815196
[4]	valid_0's rmse: 0.81466
[5]	valid_0's rmse: 0.814243
[6]	valid_0's rmse: 0.81393
[7]	valid_0's rmse: 0.813614
[8]	valid_0's rmse: 0.813404
[9]	valid_0's rmse: 0.813235
[10]	valid_0's rmse: 0.813094
[11]	valid_0's rmse: 0.812907
[12]	valid_0's rmse: 0.812761
[13]	valid_0's rmse: 0.812663
[14]	valid_0's rmse: 0.812535
[15]	valid_0's rmse: 0.812471
[16]	valid_0's rmse: 0.812425
[17]	valid_0's rmse: 0.812343
[18]	valid_0's rmse: 0.812285
[19]	valid_0's rmse: 0.812216
[20]	valid_0's rmse: 0.812164
[21]	valid_0's rmse: 0.812144
[22]	valid_0's rmse: 0.812097
[23]	valid_0's rmse: 0.812046
[24]	valid_0's rmse: 0.812016
[25]	valid_0's rmse: 0.811988
[26]	valid_0's rmse: 0.811976
[27]	valid_0's rmse: 0.811961
[28]	valid_0's rmse: 0.81194
[29]	valid_0's rmse: 0.811935
[30]	valid_0's rmse: 0.811925
[31]	valid_0's rmse: 0.811916
[32]	valid_0's rmse: 

100%|██████████| 100/100 [04:18<00:00,  2.59s/it]

[I 2022-03-28 12:50:49,351] Trial 119 finished with value: 0.8116290459571452 and parameters: {'boosting': 'gbdt', 'learning_rate': 0.14978024133215392, 'max_depth': 3, 'bagging_fraction': 0.7506254268378216, 'bagging_freq': 38, 'lambda_l1': 4.708023037316338, 'lambda_l2': 19.261135303814246}. Best is trial 30 with value: 0.8115003026285839.


In [10]:
print(study.best_params)
print(study.best_trial.user_attrs)
#ver1
# {'bagging_fraction': 0.9385460230532178, 'bagging_freq': 19, 'boosting': 'gbdt', 'lambda_l1': 0.23876030639601029, 'lambda_l2': 1.9250936095498616, 'learning_rate': 0.38076969093294494, 'max_depth': 2}
# {'best_iteration': 44, 'train_rmse': 0.47535805047080326, 'val_rmse': 0.4801005424536416}

#ver2
# {'bagging_fraction': 0.9203799956978648, 'bagging_freq': 23, 'boosting': 'gbdt', 'lambda_l1': 0.03864251625353562, 'lambda_l2': 0.021763021963546413, 'learning_rate': 0.16348254024089015, 'max_depth': 3}
# {'best_iteration': 55, 'train_rmse': 0.47376841163959976, 'val_rmse': 0.4797017907644061}

{'bagging_fraction': 0.9450872427898119, 'bagging_freq': 25, 'boosting': 'gbdt', 'lambda_l1': 0.00030699310765714517, 'lambda_l2': 1.428676378579658, 'learning_rate': 0.08054623045571584, 'max_depth': 5}
{'best_iteration': 96, 'train_rmse': 0.8065061559824701, 'val_rmse': 0.8115003026285839}


In [11]:
trials_df = study.trials_dataframe()
print(trials_df.shape)
trials_df.tail(20)

(120, 16)


,number,value,datetime_start,datetime_complete,duration,params_bagging_fraction,params_bagging_freq,params_boosting,params_lambda_l1,params_lambda_l2,params_learning_rate,params_max_depth,user_attrs_best_iteration,user_attrs_train_rmse,user_attrs_val_rmse,state
100,100,0.811578,2022-03-28 12:49:57.693235,2022-03-28 12:50:02.200071,0 days 00:00:04.506836,0.860553,37,gbdt,1.548901,1.931595,0.051875,3,445,0.807671,0.811578,COMPLETE
101,101,0.811577,2022-03-28 12:50:02.236201,2022-03-28 12:50:05.937403,0 days 00:00:03.701202,0.863661,40,gbdt,3.176988,1.890872,0.055069,3,358,0.807976,0.811577,COMPLETE
102,102,0.811582,2022-03-28 12:50:05.974817,2022-03-28 12:50:09.286709,0 days 00:00:03.311892,0.845677,38,gbdt,4.425284,3.367501,0.059903,3,331,0.808005,0.811582,COMPLETE
103,103,0.811657,2022-03-28 12:50:09.322815,2022-03-28 12:50:11.527644,0 days 00:00:02.204829,0.796367,37,gbdt,1.606287,1.690579,0.118658,3,200,0.807574,0.811657,COMPLETE
104,104,0.812014,2022-03-28 12:50:11.565045,2022-03-28 12:50:12.336737,0 days 00:00:00.771692,0.865062,40,gbdt,3.499869,1.162825,0.989384,3,10,0.809194,0.812014,COMPLETE
105,105,0.811601,2022-03-28 12:50:12.372146,2022-03-28 12:50:15.261182,0 days 00:00:02.889036,0.811275,37,gbdt,1.871493,8.782644,0.051747,3,283,0.808384,0.811601,COMPLETE
106,106,0.811702,2022-03-28 12:50:15.296438,2022-03-28 12:50:16.756589,0 days 00:00:01.460151,0.732824,35,gbdt,3.238842,2.146256,0.207268,3,115,0.807681,0.811702,COMPLETE
107,107,0.811572,2022-03-28 12:50:16.794155,2022-03-28 12:50:19.621406,0 days 00:00:02.827251,0.829449,40,gbdt,9.749267,4.182765,0.074805,3,276,0.808007,0.811572,COMPLETE
108,108,0.811532,2022-03-28 12:50:19.655696,2022-03-28 12:50:22.544392,0 days 00:00:02.888696,0.834528,40,gbdt,9.810009,4.204242,0.079306,3,281,0.807920,0.811532,COMPLETE
109,109,0.811515,2022-03-28 12:50:22.579858,2022-03-28 12:50:25.454950,0 days 00:00:02.875092,0.833341,40,gbdt,9.634989,6.914819,0.076940,3,278,0.807962,0.811515,COMPLETE


In [12]:
params = study.best_params
params['num_iterations'] = 96
params['early_stopping_round'] = 50
params['metric'] = 'rmse'

lgb_model_selected = lgb.train(params=params, train_set=d_train, valid_sets=[d_val],)

C:\Users\Haripriya\anaconda3\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.068504 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4284
[LightGBM] [Info] Number of data points in the train set: 382992, number of used features: 209
[LightGBM] [Info] Start training from score 1.463117
[1]	valid_0's rmse: 0.816947
Training until validation scores don't improve for 50 rounds
[2]	valid_0's rmse: 0.816279
[3]	valid_0's rmse: 0.815705
[4]	valid_0's rmse: 0.815234
[5]	valid_0's rmse: 0.814819
[LightGBM] [Warning] No further splits with positive gain

In [13]:
lgb_model_selected.best_iteration

96

In [14]:
train_predict = lgb_model_selected.predict(X_train, num_iteration=lgb_model_selected.best_iteration)
val_predict = lgb_model_selected.predict(X_val, num_iteration=lgb_model_selected.best_iteration)

train_rmse = np.sqrt(mean_squared_error(y_train, train_predict))
val_rmse = np.sqrt(mean_squared_error(y_val, val_predict))

train_rmse, val_rmse

(0.8065061559824701, 0.8115003026285839)

In [ ]:
val_predict_xgb_ver2 =  np.load('data_feather/Val_predict_xgb_ver2.npy', allow_pickle=True)
val_predict_xgb_ver2.shape, val_predict.shape

In [ ]:
val_predict_xgb_ver3 =  np.load('data_feather/Val_predict_xgb_ver3.npy', allow_pickle=True)
val_predict_xgb_ver3.shape, val_predict.shape

In [15]:
val_predict_xgb_ver4 =  np.load('data_out/results/Val_predict_xgb_ver4.npy', allow_pickle=True)
val_predict_xgb_ver4.shape, val_predict.shape

((19125,), (95749,))

In [18]:
val_predict_xgb_ver5 =  np.load('data_feather/Val_predict_xgb_ver5.npy', allow_pickle=True)
val_predict_xgb_ver5.shape, val_predict.shape

((95749,), (95749,))

In [19]:
for i in np.arange(0,2,0.1):
    j = 2-i
    val_predict_full = np.mean([i*val_predict,j *val_predict_xgb_ver5], axis = 0)
    val_full_rmse = np.sqrt(mean_squared_error(y_val, val_predict_full))
    print(i,j,val_full_rmse)

0.0 2.0 0.8115285228484108
0.1 1.9 0.8115161672559391
0.2 1.8 0.8115049639483912
0.30000000000000004 1.7 0.8114949121177866
0.4 1.6 0.8114860128292115
0.5 1.5 0.8114782648357407
0.6000000000000001 1.4 0.8114716692778401
0.7000000000000001 1.2999999999999998 0.8114662259322085
0.8 1.2 0.8114619342085349
0.9 1.1 0.8114587950989153
1.0 1.0 0.8114568079591267
1.1 0.8999999999999999 0.8114559729545237
1.2000000000000002 0.7999999999999998 0.8114562901299692
1.3 0.7 0.8114577591992606
1.4000000000000001 0.5999999999999999 0.8114603805024813
1.5 0.5 0.8114641541067614
1.6 0.3999999999999999 0.8114690797583074
1.7000000000000002 0.2999999999999998 0.8114751573200487
1.8 0.19999999999999996 0.8114823871417451
1.9000000000000001 0.09999999999999987 0.8114907688894746


In [20]:
val_predict_full = np.mean([1.1*val_predict,0.9 *val_predict_xgb_ver4], axis = 0)
val_full_rmse = np.sqrt(mean_squared_error(y_val, val_predict_full))
print(i,j,val_full_rmse)

1.9000000000000001 0.09999999999999987 0.8114559729545237


In [ ]:
val_predict_full[:10]

In [22]:
# test_predict_df_xgb = pd.read_csv('data_out/my_submission_9.csv')
# test_predict_df_xgb = pd.read_csv('data_out/my_submission_2.csv')
test_predict_df_xgb = pd.read_csv('data_out/my_submission_xgb_13.csv')
test_predict_df_xgb.head()

,postcode,Accident_risk_index
0,AB10 1AU,1.420454
1,AB10 1PG,1.388909
2,AB10 1TT,1.564415
3,AB10 1YP,1.459639
4,AB10 6LQ,1.403239


In [31]:
test_df = pd.read_feather('data_feather/test_df_processed_2.feather')

In [26]:
test_df = pd.read_csv('data_feather/test_ungrouped.csv', index_col=0)
# test_df['Did_Police_Officer_Attend_Scene_of_Accident'] = test_df['Did_Police_Officer_Attend_Scene_of_Accident'].replace({'Yes': 1, 'No' : 0})
X_test = np.load('data/X_test_ungrouped.npy', allow_pickle=True)

In [24]:
# X_test = test_df.values[:,3:]
d_test = lgb.Dataset(X_val, label=y_val, silent=True)

In [27]:
test_predict = lgb_model_selected.predict(X_test, num_iteration=lgb_model_selected.best_iteration)
# test_predict_df = pd.DataFrame(test_df.iloc[:,0])
# test_predict_df['Accident_risk_index'] = test_predict
# test_predict_df.head()
test_predict_df = pd.DataFrame()
test_predict_df['postcode'] = test_df['postcode']
test_predict_df['Accident_risk_index'] = test_predict
test_predict_df.shape

(121259, 2)

In [34]:
## Only for ungrouped
final_df = test_predict_df.groupby('postcode')['Accident_risk_index'].mean()
test_predict_df = final_df.copy()

KeyError: 'Column not found: Accident_risk_index'

In [35]:
test_predict_df = test_predict_df.reset_index()

In [36]:
test_predict_df.shape, test_predict_df_xgb.shape

((49772, 2), (49772, 2))

In [37]:
test_predict_df_full = test_predict_df.copy()

In [38]:
test_predict_df_full = test_predict_df_full.merge(test_predict_df_xgb, on = 'postcode')
test_predict_df_full.head()

,postcode,Accident_risk_index_x,Accident_risk_index_y
0,AB10 1AU,1.416221,1.420454
1,AB10 1PG,1.387861,1.388909
2,AB10 1TT,1.592677,1.564415
3,AB10 1YP,1.471452,1.459639
4,AB10 6LQ,1.406842,1.403239


In [39]:
test_predict_df_full['Accident_risk_index'] = ((1.1*test_predict_df_full['Accident_risk_index_x'] + 0.9 * test_predict_df_full['Accident_risk_index_y'])/2)
test_predict_df_full.head()

,postcode,Accident_risk_index_x,Accident_risk_index_y,Accident_risk_index
0,AB10 1AU,1.416221,1.420454,1.418126
1,AB10 1PG,1.387861,1.388909,1.388333
2,AB10 1TT,1.592677,1.564415,1.579959
3,AB10 1YP,1.471452,1.459639,1.466136
4,AB10 6LQ,1.406842,1.403239,1.405221


In [40]:
test_predict_df_full[['postcode','Accident_risk_index']].to_csv('data_out/my_submission_13.csv', index=False)